In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

class SinusoidalPosEmb(nn.Module):
    """Sinusoidal Positional Embedding for time conditioning."""
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, t):
        half_dim = self.dim // 2
        emb = torch.exp(-torch.arange(half_dim, dtype=torch.float32) * (torch.log(torch.tensor(10000.0)) / (half_dim - 1)))
        emb = t[:, None] * emb[None, :]
        return torch.cat((emb.sin(), emb.cos()), dim=-1)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.norm1 = nn.BatchNorm2d(out_channels)
        self.norm2 = nn.BatchNorm2d(out_channels)
        self.skip = nn.Conv2d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else nn.Identity()

    def forward(self, x):
        h = F.relu(self.norm1(self.conv1(x)))
        h = self.norm2(self.conv2(h))
        return F.relu(h + self.skip(x))

class DownsampleBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownsampleBlock, self).__init__()
        self.res_block = ResidualBlock(in_channels, out_channels)
        self.downsample = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.res_block(x)
        x = self.downsample(x)
        return x

class UpsampleBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpsampleBlock, self).__init__()
        self.upsample = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.res_block = ResidualBlock(out_channels, out_channels)

    def forward(self, x):
        x = self.upsample(x)
        x = self.res_block(x)
        return x

class DiffusionUNet(nn.Module):
    def __init__(self, in_channels, out_channels, base_channels=64, time_dim=128):
        super(DiffusionUNet, self).__init__()
        self.initial_conv = nn.Conv2d(in_channels, base_channels, kernel_size=3, padding=1)
        self.down1 = DownsampleBlock(base_channels, base_channels * 2)
        self.down2 = DownsampleBlock(base_channels * 2, base_channels * 4)
        self.down3 = DownsampleBlock(base_channels * 4, base_channels * 8)
        self.middle = ResidualBlock(base_channels * 8, base_channels * 8)
        self.up3 = UpsampleBlock(base_channels * 8, base_channels * 4)
        self.up2 = UpsampleBlock(base_channels * 4, base_channels * 2)
        self.up1 = UpsampleBlock(base_channels * 2, base_channels)
        self.final_conv = nn.Conv2d(base_channels, 1, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveAvgPool2d(1)  # Add global pooling

    def forward(self, x):
        x = self.initial_conv(x)
        x1 = self.down1(x)
        x2 = self.down2(x1)
        x3 = self.down3(x2)
        x_middle = self.middle(x3)
        x = self.up3(x_middle + x3)
        x = self.up2(x + x2)
        x = self.up1(x + x1)
        x = self.final_conv(x)
        x = self.pool(x)  # Pool to [batch_size, 1, 1, 1]
        return x.view(x.size(0))  # Flatten to [batch_size]


In [2]:
# Dataset placeholder class
class CASIADataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")
    
        if self.transform:
            image = self.transform(image)
    
        # Expand label to match the image dimensions if it's a binary classification map
        label = torch.tensor(label, dtype=torch.float32)
        label = label.expand((image.shape[1], image.shape[2]))  # Match height and width
    
        return image, label

In [7]:
import os
from PIL import Image

class CASIADataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Traverse through directories and collect image paths with labels
        for label, folder in enumerate(["Au", "Tp"]):
            folder_path = os.path.join(root_dir, folder)
            for filename in os.listdir(folder_path):
                if filename.endswith(('.tif', '.jpg', '.jpeg')):
                    self.image_paths.append(os.path.join(folder_path, filename))
                    self.labels.append(label)  # 0 for genuine, 1 for forged

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

# Dataset root directory (update this path)
root_dir = "C:/Users/ayush/Desktop/casia 2"

In [ ]:
model = DiffusionUNet(in_channels=3, out_channels=1, base_channels=64)  # Single scalar output per sample
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
train_dataset = CASIADataset(root_dir=root_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(10):
    model.train()
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)  # Outputs shape: [batch_size]
        labels = labels.view(-1)  # Flatten labels to [batch_size]

        loss = criterion(outputs, labels.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predictions = torch.sigmoid(outputs) > 0.5
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

    accuracy = correct / total
    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")


In [ ]:
import torch
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Define the transformation for the test dataset
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Define the test dataset root directory (update the path accordingly)
test_root_dir = "/kaggle/input/casia-2-test/casia 2 dataset test"

# Create the test dataset and DataLoader
test_dataset = CASIADataset(root_dir=test_root_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model.eval()  # Set the model to evaluation mode
all_predictions = []
all_labels = []
with torch.no_grad():  # Disable gradient calculation for testing
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        predictions = torch.sigmoid(outputs.squeeze()) > 0.5  # Apply sigmoid and threshold at 0.5

        # Collect predictions and true labels
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Convert to NumPy arrays for confusion matrix calculation
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

# Compute the confusion matrix
cm = confusion_matrix(all_labels, all_predictions)

# Plot confusion matrix
plot_confusion_matrix(cm, classes = ["Au", "Tp"])

# Print classification report
print(classification_report(all_predictions, all_labels))